In [46]:
import pandas as pd
import numpy as np
from faker import Faker
import hashlib

In [47]:
# %pip install faker

In [48]:
df = pd.read_excel("data/raw/arquivo_anonimizado.xlsx")

In [49]:
faker = Faker("pt_BR")

quantidade_de_nomes = df.shape[0]
nomes_completos = [faker.name() for _ in range(quantidade_de_nomes)]

df["nome_completo"] = nomes_completos

In [50]:
df.head(5).T

,0,1,2,3,4
Unnamed: 0,0,1,2,3,4
matriculaDRE,0,1,2,3,4
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Informática,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por abandono,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
disciplinasCursadas,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 062...,2000/1 - MAA123 Algebra para Informatica - 009...,2000/1 - MAA123 Algebra para Informatica - 040...,2000/1 - MAA123 Algebra para Informatica - 081...
crPorPeriodo,2000/1 - 7.6\n2000/2 - 7.1\n2001/1 - 5.9\n2001...,2000/1 - 7.6\n2000/2 - 6.5\n2001/1 - 5.1\n2001...,2000/1 - 3.3\n2000/2 - 4.7\n2001/1 - 0.7,2000/1 - 6.3\n2000/2 - 5.1\n2001/1 - 2.8\n2001...,2000/1 - 8.3\n2000/2 - 7\n2001/1 - 7.2\n2001/2...


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2817 entries, 0 to 2816
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 2817 non-null   int64  
 1   matriculaDRE               2817 non-null   int64  
 2   periodoIngressoUFRJ        2817 non-null   object 
 3   cursoIngressoUFRJ          2817 non-null   object 
 4   cursoAtual                 2817 non-null   object 
 5   periodoIngressoCursoAtual  2817 non-null   object 
 6   situacaoMatriculaAtual     2817 non-null   object 
 7   sexo                       2817 non-null   object 
 8   disciplinasCursadas        2650 non-null   object 
 9   crPorPeriodo               2644 non-null   object 
 10  craPorPeriodo              2642 non-null   object 
 11  periodosIntegralizados     2817 non-null   int64  
 12  cargaHorariaAcumulada      2817 non-null   int64  
 13  CRA                        2654 non-null   float

# Processamento


Algumas colunas não trazem ganho de informação, como por exemplo reprovações já estão na coluna disciplinas cursadas, e periodo com cr menor que 3, pode ser analisar na coluna CR por periodo


In [52]:
df.drop(
    [
        "Unnamed: 0",
        "CRA",
        "periodosCRMenor3",
        "periodosTrancados",
        "periodosCancelados",
        "reprovacoes",
    ],
    axis=1,
    inplace=True,
)

Removendo alunos que não possui disciplinas cursas, pois se não há disciplinas, não há metricas a serem analisadas.


In [53]:
df.dropna(subset=["disciplinasCursadas"], inplace=True)

## Normaliazndo Colunas Multivaloradas


### CR por Periodo


In [54]:
df["crPorPeriodo"] = df["crPorPeriodo"].astype(str)
df["crPorPeriodo"] = df["crPorPeriodo"].apply(
    lambda x: [tuple(item.split(" - ")) for item in x.split("\n")]
)
df_exploded = df.explode("crPorPeriodo")
df_exploded.head(5).T

,0,0,0,0,0
matriculaDRE,0,0,0,0,0
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
disciplinasCursadas,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...
crPorPeriodo,"(2000/1, 7.6)","(2000/2, 7.1)","(2001/1, 5.9)","(2001/2, 6.8)","(2002/1, 6.3)"
craPorPeriodo,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...


In [55]:
df_crPeriodo = pd.DataFrame(
    df_exploded["crPorPeriodo"].tolist(),
    index=df_exploded.index,
    columns=("periodo", "cr_periodo"),
)
df_crPeriodo["matriculaDRE"] = df_exploded["matriculaDRE"]
df_crPeriodo

,periodo,cr_periodo,matriculaDRE
0,2000/1,7.6,0
0,2000/2,7.1,0
0,2001/1,5.9,0
0,2001/2,6.8,0
0,2002/1,6.3,0
...,...,...,...
2775,2022/1,7.5,2775
2776,2022/1,7.8,2776
2777,2022/1,7.9,2777
2778,2022/1,6.7,2778


### CRA por Periodo


In [56]:
df["craPorPeriodo"] = df["craPorPeriodo"].astype(str)
df["craPorPeriodo"] = df["craPorPeriodo"].apply(
    lambda x: [tuple(item.split(" - ")) for item in x.split("\n")]
)
df_exploded = df.explode("craPorPeriodo")
df_exploded.head(5).T

,0,0,0,0,0
matriculaDRE,0,0,0,0,0
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
disciplinasCursadas,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...
crPorPeriodo,"[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ..."
craPorPeriodo,"(2000/1, 7.6)","(2000/2, 7.4)","(2001/1, 6.9)","(2001/2, 6.9)","(2002/1, 6.8)"


In [57]:
df_craPeriodo = pd.DataFrame(
    df_exploded["craPorPeriodo"].tolist(),
    index=df_exploded.index,
    columns=("periodo", "cr_acumulado"),
)
df_craPeriodo["matriculaDRE"] = df_exploded["matriculaDRE"]
df_craPeriodo

,periodo,cr_acumulado,matriculaDRE
0,2000/1,7.6,0
0,2000/2,7.4,0
0,2001/1,6.9,0
0,2001/2,6.9,0
0,2002/1,6.8,0
...,...,...,...
2775,2022/1,7.5,2775
2776,2022/1,7.8,2776
2777,2022/1,7.9,2777
2778,2022/1,6.7,2778


### Disciplinas Cursadas


Nessa seção vamos separar a lista de disciplinas em varias colunas, uma para cada disciplina


In [58]:
def processar_disciplinas(dados):
    dados = dados.astype(str)
    return dados.apply(lambda x: [split_disciplina(item) for item in x.split("\n")])


def split_disciplina(disciplina):
    detalhes_disciplinas = disciplina.split(" - ")
    if len(detalhes_disciplinas) == 5:
        detalhes_disciplinas[1] = " - ".join(detalhes_disciplinas[1:3])
        del detalhes_disciplinas[2]
    return tuple(detalhes_disciplinas)

In [59]:
df["disciplinasCursadas"] = df["disciplinasCursadas"].astype(str)
# df['disciplinasCursadas'] = df['disciplinasCursadas'].apply(lambda x: [tuple(item.split(' - ')) for item in x.split('\n')])
df["disciplinasCursadas"] = processar_disciplinas(df["disciplinasCursadas"])

In [60]:
df_exploded = df.explode("disciplinasCursadas")
df_exploded.head(5).T

,0,0,0,0,0
matriculaDRE,0,0,0,0,0
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
disciplinasCursadas,"(2000/1, MAA123 Algebra para Informatica, 060,...","(2000/1, MAB111 Fund da Computação Digital, 08...","(2000/1, MAB120 Computacao para Informatica, 0...","(2000/1, MAE111 Cálculo Infinitesimal I, 056, ...","(2000/1, MAE115 Cálculo Vetorial e G Analitica..."
crPorPeriodo,"[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ..."
craPorPeriodo,"[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ...","[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ...","[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ...","[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ...","[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ..."


Agora vamos pegar as informações de cada desciplina e desmembrar em novas colunas


In [61]:
df_exploded["disciplinasCursadas"] = df_exploded["disciplinasCursadas"].apply(
    lambda x: (
        x[0].replace(" ", ""),
        x[1].split()[0].replace(" ", ""),
        " ".join(x[1].split()[1:]),
        x[2].replace(" ", ""),
        x[3].replace(" ", ""),
    )
)

In [62]:
df_disciplina = pd.DataFrame(
    df_exploded["disciplinasCursadas"].tolist(),
    index=df_exploded.index,
    columns=(
        "periodo",
        "cod_disciplina",
        "disciplina",
        "grau_disciplina",
        "situacao_disciplina",
    ),
)
df_disciplina["matriculaDRE"] = df_exploded["matriculaDRE"]
df_disciplina

,periodo,cod_disciplina,disciplina,grau_disciplina,situacao_disciplina,matriculaDRE
0,2000/1,MAA123,Algebra para Informatica,060,Aprovado,0
0,2000/1,MAB111,Fund da Computação Digital,089,Aprovado,0
0,2000/1,MAB120,Computacao para Informatica,094,Aprovado,0
0,2000/1,MAE111,Cálculo Infinitesimal I,056,Aprovado,0
0,2000/1,MAE115,Cálculo Vetorial e G Analitica,085,Aprovado,0
...,...,...,...,...,...,...
2779,2022/1,ICP111,Fund Computação Digital,067,Aprovado,2779
2779,2022/1,ICP112,Sistemas de Informação,082,Aprovado,2779
2779,2022/1,ICP120,Computação I (CC),075,Aprovado,2779
2779,2022/1,ICP136,Introd Pensamento Dedutivo,081,Aprovado,2779


## Meger Colunas Tratadas


In [63]:
df_merge = pd.merge(
    df_disciplina, df_craPeriodo, on=["periodo", "matriculaDRE"], how="left"
)
df_merge = pd.merge(df_merge, df_crPeriodo, on=["periodo", "matriculaDRE"], how="left")
df_merge

,periodo,cod_disciplina,disciplina,grau_disciplina,situacao_disciplina,matriculaDRE,cr_acumulado,cr_periodo
0,2000/1,MAA123,Algebra para Informatica,060,Aprovado,0,7.6,7.6
1,2000/1,MAB111,Fund da Computação Digital,089,Aprovado,0,7.6,7.6
2,2000/1,MAB120,Computacao para Informatica,094,Aprovado,0,7.6,7.6
3,2000/1,MAE111,Cálculo Infinitesimal I,056,Aprovado,0,7.6,7.6
4,2000/1,MAE115,Cálculo Vetorial e G Analitica,085,Aprovado,0,7.6,7.6
...,...,...,...,...,...,...,...,...
98634,2022/1,ICP111,Fund Computação Digital,067,Aprovado,2779,7.8,7.8
98635,2022/1,ICP112,Sistemas de Informação,082,Aprovado,2779,7.8,7.8
98636,2022/1,ICP120,Computação I (CC),075,Aprovado,2779,7.8,7.8
98637,2022/1,ICP136,Introd Pensamento Dedutivo,081,Aprovado,2779,7.8,7.8


In [64]:
df_completo = pd.merge(df_merge, df, on=["matriculaDRE"], how="left")
df_completo.sample(5).T

,27449,73054,55468,85676,64604
periodo,2005/1,2016/0,2018/2,2018/2,2015/2
cod_disciplina,FIT111,MAE993,MABX02,MAC238,MAE994
disciplina,Física I,Cálculo Int e Diferencial III,Projeto Final de Curso,Cálculo Diferen e Integral III,Cálculo Int e Diferencial IV
grau_disciplina,073,T,000,064,058
situacao_disciplina,Aprovado,Aprovado,Reprovadomedia,Aprovado,Aprovado
matriculaDRE,630,1778,1335,2137,1565
cr_acumulado,7.8,NaN,5.6,5.9,4.7
cr_periodo,7.6,NaN,0,6.3,5.8
periodoIngressoUFRJ,2004/2,2013/2,2010/1,2016/2,2012/1
cursoIngressoUFRJ,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Ciências Matemáticas e da Terra,Bacharelado em Ciência da Computação


In [65]:
df_completo[["ano", "semestre"]] = df_completo["periodo"].str.split("/", expand=True)
df_completo.head(5).T

,0,1,2,3,4
periodo,2000/1,2000/1,2000/1,2000/1,2000/1
cod_disciplina,MAA123,MAB111,MAB120,MAE111,MAE115
disciplina,Algebra para Informatica,Fund da Computação Digital,Computacao para Informatica,Cálculo Infinitesimal I,Cálculo Vetorial e G Analitica
grau_disciplina,060,089,094,056,085
situacao_disciplina,Aprovado,Aprovado,Aprovado,Aprovado,Aprovado
matriculaDRE,0,0,0,0,0
cr_acumulado,7.6,7.6,7.6,7.6,7.6
cr_periodo,7.6,7.6,7.6,7.6,7.6
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática


In [66]:
df_completo.drop(
    ["disciplinasCursadas", "crPorPeriodo", "craPorPeriodo"], axis=1, inplace=True
)

### Padronizando Nomeclatura das Colunas


In [67]:
colunas = {
    "cod_disciplina": "cod_disciplina",
    "disciplina": "nome_disciplina",
    "matriculaDRE": "matricula_dre",
    "periodoIngressoUFRJ": "periodo_ingresso_ufrj",
    "cursoIngressoUFRJ": "curso_ingresso_ufrj",
    "cursoAtual": "curso_atual",
    "situacaoMatriculaAtual": "situacao_matricula",
    "periodoIngressoCursoAtual": "periodo_ingresso_curso_atual",
    "periodosIntegralizados": "periodos_integralizados",
    "cargaHorariaAcumulada": "carga_horaria_acumulada",
}
df_completo.rename(colunas, axis=1, inplace=True)

In [68]:
df_completo.rename(columns=colunas, inplace=True)

# Realizando Cagas das Tabelas


## Dimenssão Aluno


In [69]:
colunas_aluno = ["sk_d_aluno", "nome_completo", "matricula_dre", "sexo"]

In [70]:
d_aluno = df_completo[colunas_aluno[1:]]

# d_aluno.groupby(['nome_completo', 'matricula_dre', 'sexo']).first().reset_index()
d_aluno = d_aluno.drop_duplicates().reset_index(drop=True)
d_aluno

,nome_completo,matricula_dre,sexo
0,Ana das Neves,0,M
1,Pedro Henrique Santos,1,M
2,Daniel Correia,2,M
3,Joaquim Fernandes,3,M
4,Cauã Moreira,4,M
...,...,...,...
2645,Matheus Aragão,2775,M
2646,Joaquim Gonçalves,2776,M
2647,Ana Julia Nunes,2777,M
2648,Bryan Farias,2778,M


In [71]:
d_aluno["sk_d_aluno"] = d_aluno.apply(
    lambda x: hashlib.sha256(str(x["matricula_dre"]).encode()).hexdigest(), axis=1
)
d_aluno = d_aluno.reindex(columns=colunas_aluno)
d_aluno.to_csv("data/refined/d_aluno.csv", index=False)
d_aluno

,sk_d_aluno,nome_completo,matricula_dre,sexo
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,Ana das Neves,0,M
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,Pedro Henrique Santos,1,M
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,Daniel Correia,2,M
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,Joaquim Fernandes,3,M
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,Cauã Moreira,4,M
...,...,...,...,...
2645,3da16474e33b330b71e5b562cfcf7939bde3c12a08b746...,Matheus Aragão,2775,M
2646,9b3cea3bd8789ba47cc969bff5914da7ebac8e0052e282...,Joaquim Gonçalves,2776,M
2647,5a48eed290f62c93553855c36c964e1ef16603d23dcce3...,Ana Julia Nunes,2777,M
2648,a2541e84e912f4f5dd32ce131abdb6b1fdc9cf315790f1...,Bryan Farias,2778,M


## Dimenssão Disciplina


In [72]:
# def criar_dimenssao(nome_dimenssao, colunas_dimenssao, hash):
#     dimenssao = df_completo[colunas_dimenssao[1:]]
#     dimenssao = dimenssao.drop_duplicates().reset_index(drop=True)
#     dimenssao[f'sk_d_{nome_dimenssao}'] = d_disciplina.apply(lambda x: hashlib.sha256(str(x[hash]).encode()).hexdigest(), axis=1)
#     d_disciplina = d_disciplina.reindex(columns=colunas_aluno)
#     d_disciplina.to_csv(f'data/d_{nome_dimenssao}.csv', index=False)
#     d_disciplina

In [73]:
colunas_disciplina = ["sk_d_disciplina", "cod_disciplina", "nome_disciplina"]

In [74]:
d_disciplina = df_completo[colunas_disciplina[1:]]
d_disciplina = d_disciplina.drop_duplicates().reset_index(drop=True)
d_disciplina

,cod_disciplina,nome_disciplina
0,MAA123,Algebra para Informatica
1,MAB111,Fund da Computação Digital
2,MAB120,Computacao para Informatica
3,MAE111,Cálculo Infinitesimal I
4,MAE115,Cálculo Vetorial e G Analitica
...,...,...
1589,NEP144,Tóp Esp Polít Públ Dir Hum VI
1590,NEP142,Tóp Esp Polít Públ Dir Hum IV
1591,MAE245,Álgebra Linear II - EQ
1592,ICP114,Computação I Ep


In [75]:
d_disciplina["sk_d_disciplina"] = d_disciplina.apply(
    lambda x: hashlib.sha256(
        str(x["cod_disciplina"] + x["nome_disciplina"]).encode()
    ).hexdigest(),
    axis=1,
)
d_disciplina = d_disciplina.reindex(columns=colunas_disciplina)
d_disciplina.to_csv("data/refined/d_disciplina.csv", index=False)
d_disciplina

,sk_d_disciplina,cod_disciplina,nome_disciplina
0,26ae25e81ccc85aecdad201a410a09485d415acd4daf6e...,MAA123,Algebra para Informatica
1,4ba51a95ba6087306319254f45d5ee5ad335922069282f...,MAB111,Fund da Computação Digital
2,09e5060ee94ea2b9604e23295927e68d40d481a362d81e...,MAB120,Computacao para Informatica
3,63637e47a78400c87bb1b82da2fc5b6c00e78c0c77c779...,MAE111,Cálculo Infinitesimal I
4,6f904cf605935af92e93713c9eead620c9c92cd971dfc4...,MAE115,Cálculo Vetorial e G Analitica
...,...,...,...
1589,f711fb63fc524f5758c111314b49b44d86a21e48bb458e...,NEP144,Tóp Esp Polít Públ Dir Hum VI
1590,eac458f82acdd92fd7aa393c4453804d5516adbbaa6158...,NEP142,Tóp Esp Polít Públ Dir Hum IV
1591,574c8aa6dc1f57bf5ca26827a0871cbd70ef3416288591...,MAE245,Álgebra Linear II - EQ
1592,9d40b24dedf530a6991c38a516e1275f152e62264a656e...,ICP114,Computação I Ep


## Dimenssão Periodo


In [76]:
colunas_periodo = ["sk_d_periodo", "ano", "semestre"]

In [77]:
d_periodo = df_completo[colunas_periodo[1:]]
d_periodo = d_periodo.drop_duplicates().reset_index(drop=True)
d_periodo

,ano,semestre
0,2000,1
1,2000,2
2,2001,1
3,2001,2
4,2002,1
...,...,...
67,2019,0
68,2021,0
69,2013,3
70,2014,3


In [78]:
d_periodo["sk_d_periodo"] = d_periodo.apply(
    lambda x: hashlib.sha256(str(x["ano"] + x["semestre"]).encode()).hexdigest(), axis=1
)
d_periodo = d_periodo.reindex(columns=colunas_periodo)
d_periodo.to_csv("data/refined/d_periodo.csv", index=False)
d_periodo

,sk_d_periodo,ano,semestre
0,fc82267b45dcbf8db9e4ec10055373cfff41efcbc5b83a...,2000,1
1,29433eae6f7f1308d9799275f3a90a0afe1fef0e1818a7...,2000,2
2,e4b8f2da99fd9407c758cd306f846562424ec5c38d3202...,2001,1
3,b6167a59dbfdca3d716a326a015dd8a00f417590eb34e2...,2001,2
4,c291fe829d40dc63fc4fa578f6208f425064945d31ca82...,2002,1
...,...,...,...
67,8ad00ecd5eb3a6ca42f90cc776b009968ec5865f2d37b3...,2019,0
68,fa122eae077795ac4ff0f80bb75550a9a2c30a192d035c...,2021,0
69,fa103adf3e582291297b8330060eace6bc1a14f8737cf2...,2013,3
70,10071c57cf3b2135572815c0243a079fc18936a1259c3d...,2014,3


## Dimenssão Curso


In [79]:
colunas_curso = ["sk_d_curso", "curso_ingresso_ufrj", "curso_atual"]

In [80]:
d_curso = df_completo[colunas_curso[1:]]
d_curso = d_curso.drop_duplicates().reset_index(drop=True)
d_curso

,curso_ingresso_ufrj,curso_atual
0,Bacharelado em Informática,Bacharelado em Ciência da Computação
1,Bacharelado em Informática,Bacharelado em Informática
2,Bacharelado em Informática,Licenciatura em Matemática
3,Bacharelado em Informática,Mat Aplicada - Computação Científica
4,Matemática,Bacharelado em Ciência da Computação
...,...,...
73,Licenciatura em Física,Bacharelado em Ciência da Computação
74,Bacharelado em Ciência da Computação,Licenciatura em Geografia
75,Ciências Matemáticas e da Terra,Ciência da Computação
76,Bacharelado em Ciência da Computação,Engenharia Eletrônica e de Computação


In [81]:
d_curso["sk_d_curso"] = d_curso.apply(
    lambda x: hashlib.sha256(
        str(x["curso_ingresso_ufrj"] + x["curso_atual"]).encode()
    ).hexdigest(),
    axis=1,
)
d_curso = d_curso.reindex(columns=colunas_curso)
d_curso.to_csv("data/refined/d_curso.csv", index=False)
d_curso

,sk_d_curso,curso_ingresso_ufrj,curso_atual
0,74ea6a4ffce0cb8d3013a9978cfd9db27b906f4006c1f2...,Bacharelado em Informática,Bacharelado em Ciência da Computação
1,dbde55e27224a8c832119eba0c54cb25a32bf6f6417dd2...,Bacharelado em Informática,Bacharelado em Informática
2,cb369a9ae66bcd4659f91ed5d88c2a7674ca54645d641a...,Bacharelado em Informática,Licenciatura em Matemática
3,0367c21aa27e715610d936288c8fb811fc6dd378b050b5...,Bacharelado em Informática,Mat Aplicada - Computação Científica
4,5f20305be16f0143bc9a1bcc213bc104dd93a64e5ad031...,Matemática,Bacharelado em Ciência da Computação
...,...,...,...
73,b04e50e1e81b07e4bba26b99505e3a21fd9160bdd6c664...,Licenciatura em Física,Bacharelado em Ciência da Computação
74,2d37b7f8c6308826f3cd805e314628baf4dede1e70bb0a...,Bacharelado em Ciência da Computação,Licenciatura em Geografia
75,acbc30f06ec39601955703a6f4fb05791a031a8f7755d1...,Ciências Matemáticas e da Terra,Ciência da Computação
76,a54d2fed0213e2b06b6fb51bf134c8f5920420505116c3...,Bacharelado em Ciência da Computação,Engenharia Eletrônica e de Computação


## Dimenssão Situação Academica


In [82]:
# colunas_situacao_academica = ['sk_d_situacao_academica', 'situacaoMatriculaAtual']

In [83]:
# d_situacao_academica = df_completo[colunas_situacao_academica[1:]]
# d_situacao_academica = d_situacao_academica.drop_duplicates().reset_index(drop=True)
# d_situacao_academica

In [84]:
# d_situacao_academica['sk_d_situacao_academica'] = d_situacao_academica.apply(lambda x: hashlib.sha256(str(x['situacaoMatriculaAtual']).encode()).hexdigest(), axis=1)
# d_situacao_academica = d_situacao_academica.reindex(columns=colunas_situacao_academica)
# d_situacao_academica.to_csv('data/refined/d_situacao_academica.csv', index=False)
# d_situacao_academica

## Fato Desempenho Academico


In [85]:
f_desempenho_academico = df_completo.drop(
    [
        "periodo",
        "periodo_ingresso_curso_atual",
        "periodos_integralizados",
        "carga_horaria_acumulada",
    ],
    axis=1,
)
f_desempenho_academico.head(1)

,cod_disciplina,nome_disciplina,grau_disciplina,situacao_disciplina,matricula_dre,cr_acumulado,cr_periodo,periodo_ingresso_ufrj,curso_ingresso_ufrj,curso_atual,situacao_matricula,sexo,nome_completo,ano,semestre
0,MAA123,Algebra para Informatica,060,Aprovado,0,7.6,7.6,2000/1,Bacharelado em Informática,Bacharelado em Ciência da Computação,Cancelada por conclusão de curso,M,Ana das Neves,2000,1


In [86]:
# # D_SITUACAO_ACADEMICA
# f_desempenho_academico = pd.merge(f_desempenho_academico, d_situacao_academica, on=['situacao_matricula_atual'], how='left')
# f_desempenho_academico.drop(['situacaoMatriculaAtual'], axis=1, inplace=True)
# f_desempenho_academico.sample(5).T

In [87]:
# D_PERIODO
f_desempenho_academico = pd.merge(
    f_desempenho_academico, d_periodo, on=["ano", "semestre"], how="left"
)
f_desempenho_academico.drop(["ano", "semestre"], axis=1, inplace=True)
f_desempenho_academico.sample(5).T

,45420,54532,34457,72170,84612
cod_disciplina,MAB535,MAB624,MAB624,MAB240,EEL555
nome_disciplina,Modelagem de Sistemas Inform I,Números Inteiros Criptografia,Números Inteiros Criptografia,Computação II (CC),Sistemas Lineares II
grau_disciplina,060,100,070,036,100
situacao_disciplina,Aprovado,Aprovado,Aprovado,Reprovadomedia,Aprovado
matricula_dre,1035,1308,793,1753,2103
cr_acumulado,4.6,9.6,7.1,3.6,8.5
cr_periodo,6.8,9.6,7.1,1.7,9.3
periodo_ingresso_ufrj,2008/1,2010/1,2006/1,2013/2,2016/2
curso_ingresso_ufrj,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
curso_atual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Engenharia Eletrônica e de Computação


In [88]:
# D_CURSO
f_desempenho_academico = pd.merge(
    f_desempenho_academico,
    d_curso,
    on=["curso_ingresso_ufrj", "curso_atual"],
    how="left",
)
f_desempenho_academico.drop(
    ["curso_ingresso_ufrj", "curso_atual"], axis=1, inplace=True
)
f_desempenho_academico.sample(5).T

,22648,84204,34324,34099,1348
cod_disciplina,MAB241,MAB120,MAB122,MAB513,FIT111
nome_disciplina,Computacao II,Computação I (CC),Computação Gráfica I,Informática na Administração,Física I
grau_disciplina,064,074,050,079,058
situacao_disciplina,Aprovado,Aprovado,Aprovado,Aprovado,Aprovado
matricula_dre,526,2089,789,783,25
cr_acumulado,5.7,7.2,6,8.3,6.3
cr_periodo,5.6,7.2,6.3,7.4,6
periodo_ingresso_ufrj,2003/2,2016/2,2006/1,2006/1,2000/1
situacao_matricula,Cancelada por conclusão de curso,Ativa,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,F


In [89]:
# D_ALUNO
f_desempenho_academico = pd.merge(
    f_desempenho_academico,
    d_aluno,
    on=["nome_completo", "sexo", "matricula_dre"],
    how="left",
)
f_desempenho_academico.drop(
    ["nome_completo", "sexo", "matricula_dre"], axis=1, inplace=True
)
f_desempenho_academico.sample(5).T

,32057,16360,45178,9994,92796
cod_disciplina,MAB490,MAB625,MAB605,MABX02,ICP136
nome_disciplina,Laboratório de Banco de Dados,Tópicos em Criptografia,Recuperação de Informação,Projeto Final de Curso,Introd Pensamento Dedutivo
grau_disciplina,060,060,100,100,097
situacao_disciplina,Aprovado,Aprovado,Aprovado,Aprovado,Aprovado
cr_acumulado,5.9,9,7.2,5.4,8
cr_periodo,5.2,8.7,8.7,8.7,7.9
periodo_ingresso_ufrj,2005/2,2002/2,2007/2,2001/2,2018/2
situacao_matricula,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Rematrícula por destrancamento ou descancelamento
sk_d_periodo,ed0661bd6e1c613c15349c48cc1986e69c99cac57db13a...,551f1b6da3d076f00cf0f65b0bbb08ee6e88d26efd4abf...,12ae265fcfe315a363b94cb06b5634257af6517781d59a...,b18c61c55f17af492ac0535bf17d89dfb521c9110d0cbd...,5168f166ecaf66e62ce5c859e54e06e615b86260677db2...
sk_d_curso,69f98f9a931103c9534d1881425cd98f8d920686439325...,74ea6a4ffce0cb8d3013a9978cfd9db27b906f4006c1f2...,69f98f9a931103c9534d1881425cd98f8d920686439325...,74ea6a4ffce0cb8d3013a9978cfd9db27b906f4006c1f2...,a80f5dd573274667573b525e0750e7915a3bc967190075...


In [90]:
# D_DISCIPLINA
f_desempenho_academico = pd.merge(
    f_desempenho_academico,
    d_disciplina,
    on=["cod_disciplina", "nome_disciplina"],
    how="left",
)
f_desempenho_academico.drop(["cod_disciplina", "nome_disciplina"], axis=1, inplace=True)
f_desempenho_academico.sample(5).T

,943,68521,9237,84947,22113
grau_disciplina,057,084,074,019,066
situacao_disciplina,Aprovado,Aprovado,Aprovado,Reprovadomedia,Aprovado
cr_acumulado,6.2,8.5,5.5,NaN,8.1
cr_periodo,6.6,8.2,7.1,NaN,8.3
periodo_ingresso_ufrj,2000/1,2012/2,2001/2,2016/2,2003/2
situacao_matricula,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Trancamento automático: Perigo de cancelamento...,Cancelada por conclusão de curso
sk_d_periodo,c291fe829d40dc63fc4fa578f6208f425064945d31ca82...,e319d9b68ca50f52f6dfba34b7784ba81c72646766f52c...,62032de289e313ca8bfb85bdd381bed28c75689f6fd89f...,f37f0a0a1bb32d7877f498440e10f184d8a33b5befe6ac...,0de41f48bb8492eab4bd60b26b08b5cef09b5509f65981...
sk_d_curso,74ea6a4ffce0cb8d3013a9978cfd9db27b906f4006c1f2...,69f98f9a931103c9534d1881425cd98f8d920686439325...,74ea6a4ffce0cb8d3013a9978cfd9db27b906f4006c1f2...,a80f5dd573274667573b525e0750e7915a3bc967190075...,74ea6a4ffce0cb8d3013a9978cfd9db27b906f4006c1f2...
sk_d_aluno,4ec9599fc203d176a301536c2e091a19bc852759b255bd...,64ef8cbfd4e16b46fba5c355a47cb596c58f876c7e898f...,9b871512327c09ce91dd649b3f96a63b7408ef267c8cc5...,44c59909f17c296d6f2ec4a53efac3a951add75aa67616...,4771bef2c04a34b548b77ea7581cf821152d9dea9c2c85...
sk_d_disciplina,a790a2f3d8c80fd7e382113da81d41fa6c1720185bf086...,c851d153fe7937986b93ab95203eba4ca75ad55285bd57...,accdec69de182d26d07e5a20290dadde443846dde9d218...,f815db806b0109c4e88871c1418e2ce76d58802cde3ce2...,def4682d25ac08dff9e665120fd38abc76b1eef99aece1...


In [91]:
f_desempenho_academico.drop(["periodo_ingresso_ufrj"], axis=1, inplace=True)

In [92]:
f_desempenho_academico.sample(5).T

,43759,97788,31195,25814,6523
grau_disciplina,082,095,089,067,093
situacao_disciplina,Aprovado,Aprovado,Aprovado,Aprovado,Aprovado
cr_acumulado,4.6,8.9,9.1,7.2,7.8
cr_periodo,5.6,8.4,8.8,7.2,7.4
situacao_matricula,Cancelada por conclusão de curso,Ativa,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sk_d_periodo,ed0661bd6e1c613c15349c48cc1986e69c99cac57db13a...,5168f166ecaf66e62ce5c859e54e06e615b86260677db2...,75769823db109ec97fb1f12fa46a665d0a73079b5c893f...,551f1b6da3d076f00cf0f65b0bbb08ee6e88d26efd4abf...,c2db217ea7a34191c8f55a2acd4ac37c35611dc1946a7c...
sk_d_curso,69f98f9a931103c9534d1881425cd98f8d920686439325...,a80f5dd573274667573b525e0750e7915a3bc967190075...,69f98f9a931103c9534d1881425cd98f8d920686439325...,69f98f9a931103c9534d1881425cd98f8d920686439325...,74ea6a4ffce0cb8d3013a9978cfd9db27b906f4006c1f2...
sk_d_aluno,8c9a013ab70c0434313e3e881c310b9ff24aff1075255c...,61b1dbc91985561c5af9e6e341ffa92fa3524c1e2083f3...,35c71bd7eaf4607047bb7c186d17251942204229b897e0...,a917ca757ac59f9d568616140c2f72362fc2722ab277e7...,05ada863a4cf9660fd8c68e2295f1d35b2264815f5b605...
sk_d_disciplina,b6f2d7cebc2148d3316050cfea83db5bd2c99a8473983f...,b923d2d2bd975599cb8007d7338786bc4ff9de0fbfaa7d...,bda9f2bab1bbadfd30979ee3b1078a5fced655ad7b6f5f...,63637e47a78400c87bb1b82da2fc5b6c00e78c0c77c779...,a790a2f3d8c80fd7e382113da81d41fa6c1720185bf086...


In [93]:
f_desempenho_academico.to_csv("data/refined/f_desempenho_academico.csv", index=False)

## Fato Matricula Aluno


In [181]:
f_matricula_aluno = df_completo.drop(
    [
        "cod_disciplina",
        "nome_disciplina",
        "grau_disciplina",
        "situacao_disciplina",
        "cr_acumulado",
        "cr_periodo",
    ],
    axis=1,
)
f_matricula_aluno.head(1)

,periodo,matricula_dre,periodo_ingresso_ufrj,curso_ingresso_ufrj,curso_atual,periodo_ingresso_curso_atual,situacao_matricula,sexo,periodos_integralizados,carga_horaria_acumulada,nome_completo,ano,semestre
0,2000/1,0,2000/1,Bacharelado em Informática,Bacharelado em Ciência da Computação,2000/1,Cancelada por conclusão de curso,M,13,3105,Ana das Neves,2000,1


In [182]:
f_matricula_aluno = (
    f_matricula_aluno.groupby(
        [
            "matricula_dre",
            "situacao_matricula",
            "periodos_integralizados",
            "carga_horaria_acumulada",
            "curso_ingresso_ufrj",
            "curso_atual",
        ]
    )["periodo"]
    .max()
    .reset_index()
)
f_matricula_aluno[["ano", "semestre"]] = f_matricula_aluno["periodo"].str.split("/", expand=True)

In [183]:
f_matricula_aluno.head(2)

,matricula_dre,situacao_matricula,periodos_integralizados,carga_horaria_acumulada,curso_ingresso_ufrj,curso_atual,periodo,ano,semestre
0,0,Cancelada por conclusão de curso,13,3105,Bacharelado em Informática,Bacharelado em Ciência da Computação,2006/1,2006,1
1,1,Cancelada por conclusão de curso,10,3330,Bacharelado em Informática,Bacharelado em Ciência da Computação,2004/1,2004,1


In [189]:
teste = pd.read_excel('data/raw/arquivo_anonimizado.xlsx')

In [192]:
f_matricula_aluno[f_matricula_aluno['matricula_dre'] == 1563]

,matricula_dre,situacao_matricula,periodos_integralizados,carga_horaria_acumulada,curso_ingresso_ufrj,curso_atual,periodo,ano,semestre
1461,1563,Rematrícula por destrancamento ou descancelamento,2,2850,Bacharelado em Ciência da Computação,Ciência da Computação,2021/1,2021,1


In [190]:
teste[teste['matriculaDRE'] == 1563]

,Unnamed: 0,matriculaDRE,periodoIngressoUFRJ,cursoIngressoUFRJ,cursoAtual,periodoIngressoCursoAtual,situacaoMatriculaAtual,sexo,disciplinasCursadas,crPorPeriodo,craPorPeriodo,periodosIntegralizados,cargaHorariaAcumulada,CRA,periodosTrancados,periodosCancelados,periodosCRMenor3,reprovacoes
1563,1563,1563,2012/1,Bacharelado em Ciência da Computação,Ciência da Computação,2021/1,Rematrícula por destrancamento ou descancelamento,M,2012/1 - MAB112 Sistemas de Informação - 069 -...,2012/1 - 3.5\n2012/2 - 4.9\n2013/1 - 5.1\n2013...,2012/1 - 3.5\n2012/2 - 4.1\n2013/1 - 4.4\n2013...,2,2850,5.6,2021/2,NaN,2020/2 | 2022/1,2012/1 - MAB120 Computação I (CC) - 014 - Repr...


In [194]:
df_completo[df_completo['matricula_dre'] == 1563]

,periodo,cod_disciplina,nome_disciplina,grau_disciplina,situacao_disciplina,matricula_dre,cr_acumulado,cr_periodo,periodo_ingresso_ufrj,curso_ingresso_ufrj,curso_atual,periodo_ingresso_curso_atual,situacao_matricula,sexo,periodos_integralizados,carga_horaria_acumulada,nome_completo,ano,semestre
64442,2012/1,MAB112,Sistemas de Informação,069,Aprovado,1563,3.5,3.5,2012/1,Bacharelado em Ciência da Computação,Ciência da Computação,2021/1,Rematrícula por destrancamento ou descancelamento,M,2,2850,Manuela Campos,2012,1
64443,2012/1,MAB120,Computação I (CC),014,Reprovadomedia,1563,3.5,3.5,2012/1,Bacharelado em Ciência da Computação,Ciência da Computação,2021/1,Rematrícula por destrancamento ou descancelamento,M,2,2850,Manuela Campos,2012,1
64444,2012/1,MAB624,Números Inteiros Criptografia,070,Aprovado,1563,3.5,3.5,2012/1,Bacharelado em Ciência da Computação,Ciência da Computação,2021/1,Rematrícula por destrancamento ou descancelamento,M,2,2850,Manuela Campos,2012,1
64445,2012/1,MAE111,Cálculo Infinitesimal I,000,Reprovadomedia,1563,3.5,3.5,2012/1,Bacharelado em Ciência da Computação,Ciência da Computação,2021/1,Rematrícula por destrancamento ou descancelamento,M,2,2850,Manuela Campos,2012,1
64446,2012/2,MAB113,Organização da Informação,062,Aprovado,1563,4.1,4.9,2012/1,Bacharelado em Ciência da Computação,Ciência da Computação,2021/1,Rematrícula por destrancamento ou descancelamento,M,2,2850,Manuela Campos,2012,2
64447,2012/2,MAB120,Computação I (CC),073,Aprovado,1563,4.1,4.9,2012/1,Bacharelado em Ciência da Computação,Ciência da Computação,2021/1,Rematrícula por destrancamento ou descancelamento,M,2,2850,Manuela Campos,2012,2
64448,2012/2,MAB352,Matemática Combinatória,008,Reprovadomedia,1563,4.1,4.9,2012/1,Bacharelado em Ciência da Computação,Ciência da Computação,2021/1,Rematrícula por destrancamento ou descancelamento,M,2,2850,Manuela Campos,2012,2
64449,2013/1,MAB111,Fund da Computação Digital,071,Aprovado,1563,4.4,5.1,2012/1,Bacharelado em Ciência da Computação,Ciência da Computação,2021/1,Rematrícula por destrancamento ou descancelamento,M,2,2850,Manuela Campos,2013,1
64450,2013/1,MAB354,Computadores e Sociedade,095,Aprovado,1563,4.4,5.1,2012/1,Bacharelado em Ciência da Computação,Ciência da Computação,2021/1,Rematrícula por destrancamento ou descancelamento,M,2,2850,Manuela Campos,2013,1
64451,2013/1,MAE111,Cálculo Infinitesimal I,008,Reprovadomedia,1563,4.4,5.1,2012/1,Bacharelado em Ciência da Computação,Ciência da Computação,2021/1,Rematrícula por destrancamento ou descancelamento,M,2,2850,Manuela Campos,2013,1


In [179]:
# D_ALUNO
f_matricula_aluno = pd.merge(
    f_matricula_aluno,
    d_aluno,
    on=["matricula_dre"],
    how="inner",
)
f_matricula_aluno.drop(
    ["nome_completo", "sexo", "matricula_dre"], axis=1, inplace=True
)
f_matricula_aluno.sample(2)

,situacao_matricula,periodos_integralizados,carga_horaria_acumulada,curso_ingresso_ufrj,curso_atual,periodo,ano,semestre,sk_d_aluno
1370,Trancamento Solicitado,0,3000,Matemática Aplicada,Ciência da Computação,2020/1,2020,1,4753e699ede615170936d7dd4c55a57c11c2965a2f8854...
265,Cancelamento a pedido,1,0,Bacharelado em Informática,Bacharelado em Informática,2002/1,2002,1,1e68ed4e3d58a51096a7feea3947f40debf1fd9246ec97...


In [172]:
# D_CURSO
f_matricula_aluno = pd.merge(
    f_matricula_aluno,
    d_curso,
    on=["curso_ingresso_ufrj", "curso_atual"],
    how="inner",
)
f_matricula_aluno.drop(
    ["curso_ingresso_ufrj", "curso_atual"], axis=1, inplace=True
)
f_matricula_aluno.sample(2)

,situacao_matricula,periodos_integralizados,carga_horaria_acumulada,periodo,ano,semestre,sk_d_aluno,sk_d_curso
1760,Rematrícula por destrancamento ou descancelamento,2,2850,2021/1,2021,1,e830a0d7d5f303724e6404c8f9797580b6e77e4bdab70b...,a80f5dd573274667573b525e0750e7915a3bc967190075...
790,Cancelada por conclusão de curso,11,3090,2011/1,2011,1,130790feced08212eed7d1490dd4d7abf138543be61a47...,69f98f9a931103c9534d1881425cd98f8d920686439325...


In [173]:
f_matricula_aluno.to_csv("data/refined/f_matricula_aluno.csv", index=False)

# Analise de disciplinas com nomes errados


In [288]:
d_disciplina[d_disciplina["cod_disciplina"] == "MAB613"]

,sk_d_disciplina,cod_disciplina,nome_disciplina
51,69c74b0dc4bd99f8f39728f26b5edb757d2f12f54e08ea...,MAB613,Met Desenv de Sist Informacao
118,4831abbc49e3963e443cfe77fde3e1f4966cbaa7adf118...,MAB613,Met Desenv de Sist Informação


In [289]:
d_disciplina[d_disciplina["cod_disciplina"] == "MAB601"]
# EEL410

,sk_d_disciplina,cod_disciplina,nome_disciplina
50,fa1851c7178a5ca652bdbb609397732777b444e5be4aa4...,MAB601,Top Esp em Eng de Softaware
61,e4b4dc9868db99e7a7c2f8c49d7597179bcf9697daf319...,MAB601,Top Esp em Eng de Software


In [290]:
d_disciplina[d_disciplina["cod_disciplina"].duplicated()]

,sk_d_disciplina,cod_disciplina,nome_disciplina
61,e4b4dc9868db99e7a7c2f8c49d7597179bcf9697daf319...,MAB601,Top Esp em Eng de Software
118,4831abbc49e3963e443cfe77fde3e1f4966cbaa7adf118...,MAB613,Met Desenv de Sist Informação
358,bb19397c608b8ad6e1e91f353d8aeaa3c0109a13a33ecb...,MAB603,Gestão do Conhecimento
383,58ae4c22500c1bf1a3dd156b6405367f685508d5a856ab...,MAB230,Cálculo Numérico (CC)
417,70c5438681f6efe2902a4b9e78fd4488412a85544fdf14...,MAB602,Dw no Sup à Tomada de Decisão
419,c6fdafe7054dc9166dd654d7d98ef1d8da16718f40133a...,MAB600,Dados Semi-estruturados e XML
460,ad46eeb3845a2b7de8e5ceebc4230a1850be2509cad643...,MAB532,Mineração de Dados
462,a8cff0b3f3cdc6991275d971f5fef3fdfbbf8d6393afbf...,EEH210,Engenharia e Meio Ambiente
471,6350703008dfb26f834d42c1683bf89f50315ddbea870e...,IEE304,Desenvolvimento Socioeconomico
474,310f5c6e86c7e6cbe976fa5706be1805c8d1f8b59bd6ca...,MAB120,Computação I (CC)
